<a href="https://colab.research.google.com/github/khirotaka/Swift4TF/blob/master/Colab/Swift%20for%20TensorFlow%20for%20using%20PyTorch%20users%20No1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch使いの為の Swift for TensorFlow

In [0]:
import TensorFlow

In [2]:
let seed = TensorFlow.randomSeedForTensorFlow(using: TensorFlowSeed(graph: 0, op: 0))    // Seed固定

var x: Tensor<Float> = _Raw.empty(shape: [5, 3])    // TensorFlowの低レベルAPIにアクセスするには _Raw を使う。(Raw は非推奨になった)
print(x)

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]


Swift for TensorFlowには `ShapedArray` というものがあり、`numpy.ndarray` に近いものになっている。

In [3]:
var xx: ShapedArray<Float> = ShapedArray<Float>(shape: [5, 5], repeating: 0.0)
print("Shape of xx \(xx.shape)")
print(xx)

Shape of xx [5, 5]
[[0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]


Swift for TensorFlowで、
```python
torch.randn(5, 3)
```
を行いたいのなら、次のようにすればいい。

In [4]:
var x: Tensor<Float> = Tensor<Float>(randomNormal: [5, 3], seed: seed)
print(x)

[[ 0.59503317,  -1.2243367,  -1.2307329],
 [ 0.64013857,   1.1960995,    2.365863],
 [ 0.54103607,   0.5393618,  0.27644297],
 [ -0.5943514, -0.94016635,  -0.4811743],
 [ 0.35051414,  -1.8140777,  0.38941053]]


```python
torch.zeros(5, 3)
```
や
```python
torch.ones(5, 3)
```
も以下のように書ける。

In [5]:
var x = Tensor<Float>(ones: [5, 3])
print(x)

print("----------")    // swift で "-"*50 が使えないのホント不便

var xx = Tensor<UInt8>(zeros: [5, 3])
print(xx)

[[1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0]]
----------
[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]


`zeros_like` も `ones_like` もある。

In [6]:
var tmp = ShapedArray<Float>(repeating: 0.0, shape: [5, 5])

var x = Tensor<Float>(onesLike: Tensor<Float>(tmp))
print(x)

print("----------")

var xx = Tensor<Float>(zerosLike: Tensor<Float>(tmp))
print(xx)

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0]]
----------
[[0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]


当然、
```python
torch.tensor([5.5, 3.0])
```
のような書き方も可能

In [7]:
var x = Tensor<Float>([5.5, 3])
print(x)

[5.5, 3.0]


先ほどから出ている正規分布からサンプリングした乱数以外にも

In [8]:
var y = Tensor<Float>(randomNormal: [5, 3], seed: seed)

print(y)

[[ 0.59503317,  -1.2243367,  -1.2307329],
 [ 0.64013857,   1.1960995,    2.365863],
 [ 0.54103607,   0.5393618,  0.27644297],
 [ -0.5943514, -0.94016635,  -0.4811743],
 [ 0.35051414,  -1.8140777,  0.38941053]]


## Using GPU

```swift
var x = withDevice(.gpu, 0) {
    Tensor<Float>([1, 2, 3])
}    // これでGPUに載せる事が出来る。

```

In [0]:
import Foundation    // Dateを使う為

In [10]:
let start = Date()

var x = TensorFlow.withDevice(.gpu, 0) {
    Tensor<Float>(randomNormal: [5000, 6000], seed: seed)
}

var y = TensorFlow.withDevice(.gpu, 0) {
    matmul(x, x.transposed())
}

let end = Date().timeIntervalSince(start)
print(end)

0.5399130582809448


In [11]:
let start = Date()

var x = TensorFlow.withDevice(.cpu, 0) {
    Tensor<Float>(randomNormal: [5000, 6000], seed: seed)
}

var y = TensorFlow.withDevice(.cpu, 0) {
    matmul(x, x.transposed())
}

let end = Date().timeIntervalSince(start)
print(end)

5.296565055847168


## Numpyとの互換性

### Numpy -> TensorFlow on Swift

まず、Pythonと site-packages にインストールされているPythonライブラリを使う。

In [0]:
import PythonKit

let np = Python.import("numpy")

In [13]:
Python.version    // Python のバージョン

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


```python
let np = Python.import("numpy")
```

で`numpy` が使えるようになります。
あとは、いつもPythonで書いてるように関数やオブジェクトにアクセスできます。

In [14]:
var np_a = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype: np.float32)
np_a = np_a.reshape(2, 5)
print(np_a.shape)    // Numpy オブジェクト

(2, 5)


In [15]:
var swift_a = Tensor<Float>(numpy: np_a)!    // Numpyを元に Tensor を作成。オプショナル型なので注意
print(String(describing: type(of: swift_a)))
print(swift_a.shape)

Tensor<Float>
[2, 5]


### Swift -> Numpy

In [16]:
var new_ndarray = swift_a.makeNumpyArray()    // PyTorch の numpy() と同じノリ

print(new_ndarray.shape)

(2, 5)


Pythonに内蔵されている関数 (例えば、 `type` を使いたいのなら、以下のようにすれば行ける)

In [17]:
Python.type(new_ndarray)

<class 'numpy.ndarray'>
